In [1]:
import pandas as pd
import numpy as np

df_high = pd.read_csv("high_values/top_sequences.txt", sep="\t")
df_high = df_high.drop(" ", axis=1)
df_raw = pd.read_excel("produce/ralstonia_high.xlsx", index_col="Samples")

X_right = np.array(df_raw.drop("ralstonia", axis=1).values, dtype=np.float32)
y_right = np.array(df_raw["ralstonia"].values, dtype=np.float32)

In [2]:
df_raw

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,ralstonia
Samples,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,5.462651
1,0,0,0,0,1,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,5.161621
2,0,0,0,0,0,1,1,1,1,0,...,1,0,0,0,0,0,1,0,0,5.161621
3,0,0,0,0,0,0,0,1,1,1,...,0,0,1,0,0,1,0,1,0,5.082086
4,1,1,0,0,1,0,0,0,0,1,...,1,0,0,0,0,0,0,0,0,4.684500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,1,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2.019981
162,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,1,0,2.019360
163,1,1,0,0,0,1,0,0,0,0,...,1,1,1,0,0,1,0,1,0,2.015493


In [13]:
# Functions

import copy

def get_ralstonia_value(row, X_right, model_arch):
    X_left = np.broadcast_to(np.array(row), (X_right.shape))
    if model_arch == "siamese":
        y_pred = model.predict([X_left, X_right])
    elif model_arch == "twin":
        X_pred = np.concatenate((X_left, X_right), axis=1)
        y_pred = model.predict(X_pred)
    y_pred = y_pred.squeeze()
    y_abs = np.mean(y_pred + y_right)
    return y_abs

def perturb(row, X_right, model_arch, fork, high_val):
    row_new = copy.deepcopy(row)
    ralst_vals = []
    for index in range(len(row_new)):
        row_new[index] = 1-int(row_new[index])
        new_val = get_ralstonia_value(row_new, X_right, model_arch)
        ralst_vals.append(new_val)
        row_new = copy.deepcopy(row)
        
    best_row = copy.deepcopy(row)
    best_row[np.argmax(ralst_vals)] = 1-best_row[np.argmax(ralst_vals)]
    ralst_best_row = get_ralstonia_value(best_row, X_right, model_arch)
    with open("{}.txt".format(fork), "a") as f:
        for item in best_row:
            f.write("{} ".format(item))
        f.write("{}\n".format(ralst_best_row))
    
    if ralst_best_row > high_val:
        perturb(best_row, X_right, model_arch, fork, ralst_best_row)
    else:
        print("Converged")

In [14]:
# Perturb for siamese fork1
import tensorflow as tf
from tensorflow.keras.models import load_model

model_arch = 'siamese'
fork = "fork1_tf"
model = load_model("produce/saved_models_tf/tf_fork1.h5")
high_val = 0
for i in range(df_high.shape[0]):
    row = df_high.iloc[i].values.tolist()
    perturb(row, X_right, model_arch, fork, high_val)
    
    

2023-03-15 18:20:12.680887: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2023-03-15 18:20:12.681112: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2803200000 Hz
2023-03-15 18:20:12.711188: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged


In [15]:
# Perturb for siamese fork1
import tensorflow as tf
from tensorflow.keras.models import load_model

model_arch = 'siamese'
fork = "fork2_tf"
model = load_model("produce/saved_models_tf/tf_fork2.h5")
high_val = 0
for i in range(df_high.shape[0]):
    row = df_high.iloc[i].values.tolist()
    perturb(row, X_right, model_arch, fork, high_val)
    
    

Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged


In [16]:
# Perturb for siamese fork1
import tensorflow as tf
from tensorflow.keras.models import load_model

model_arch = 'twin'
fork = "fork3_tf"
model = load_model("produce/saved_models_tf/tf_fork3.h5")
high_val = 0
for i in range(df_high.shape[0]):
    row = df_high.iloc[i].values.tolist()
    perturb(row, X_right, model_arch, fork, high_val)
    
    

Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged


In [17]:
# Perturb for siamese fork1
import tensorflow as tf
from tensorflow.keras.models import load_model

model_arch = 'twin'
fork = "fork4_tf"
model = load_model("produce/saved_models_tf/tf_fork4.h5")
high_val = 0
for i in range(df_high.shape[0]):
    row = df_high.iloc[i].values.tolist()
    perturb(row, X_right, model_arch, fork, high_val)
    
    

Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
Converged
